In [1]:
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
import pandas as pd

In [3]:
!head usgs_fault -n 12

#Total number of fault_segments=     1
#Fault_segment =   1 nx(Along-strike)=  25 Dx= 12.00km ny(downdip)=  18 Dy=  8.80km
#Boundary of Fault_segment     1. EQ in cell 9,10. Lon: -71.6728   Lat: -31.5952
#Lon.  Lat.  Depth
      -72.60630      -32.42690        2.06900
      -72.27690      -29.75720        2.06900
      -70.72070      -29.89650       53.06600
      -71.05010      -32.56620       53.06600
      -72.60630      -32.42690        2.06900
#Lat. Lon. depth slip rake strike dip t_rup t_ris t_fal mo
     -32.426900     -72.606300       2.069000      58.115470      71.387770       6.000000      19.000000      82.800000       3.200000       1.600000   5.832598e+24
     -32.319500     -72.593100       2.069000      86.756260      92.165860       6.000000      19.000000      61.600000      11.200000       6.400000   8.707055e+24


In [11]:
df = pd.read_csv('usgs.txt', delim_whitespace=True, comment='#', skiprows=9, names='Lat. Lon. depth slip rake strike dip t_rup t_ris t_fal mo'.split(' '))

In [12]:
df.head()

,Lat.,Lon.,depth,slip,rake,strike,dip,t_rup,t_ris,t_fal,mo
0,-32.4269,-72.6063,2.069,58.11547,71.38777,6.0,19.0,82.8,3.2,1.6,5.832598e+24
1,-32.3195,-72.5931,2.069,86.75626,92.16586,6.0,19.0,61.6,11.2,6.4,8.707055e+24
2,-32.2121,-72.5798,2.069,135.00011,105.43640,6.0,19.0,42.0,1.6,1.6,1.354892e+25
3,-32.1047,-72.5666,2.069,193.42450,81.69747,6.0,19.0,52.4,1.6,4.8,1.941251e+25
4,-31.9973,-72.5533,2.069,258.40659,90.97316,6.0,19.0,50.4,6.4,1.6,2.593426e+25


In [13]:
df = df.rename(columns={
    'Lat.': 'lat',
    'Lon.': 'lon'
})

df['L'] = 12000
df['W'] = 8800
df['depth']*=1000
df['reference'] = 'center'
df['slip'] *= 0.01
df.head()

,lat,lon,depth,slip,rake,strike,dip,t_rup,t_ris,t_fal,mo,L,W,reference
0,-32.4269,-72.6063,2069.0,0.581155,71.38777,6.0,19.0,82.8,3.2,1.6,5.832598e+24,12000,8800,center
1,-32.3195,-72.5931,2069.0,0.867563,92.16586,6.0,19.0,61.6,11.2,6.4,8.707055e+24,12000,8800,center
2,-32.2121,-72.5798,2069.0,1.350001,105.43640,6.0,19.0,42.0,1.6,1.6,1.354892e+25,12000,8800,center
3,-32.1047,-72.5666,2069.0,1.934245,81.69747,6.0,19.0,52.4,1.6,4.8,1.941251e+25,12000,8800,center
4,-31.9973,-72.5533,2069.0,2.584066,90.97316,6.0,19.0,50.4,6.4,1.6,2.593426e+25,12000,8800,center


In [14]:
df.to_csv('earthquake.csv',index=False)

# Geoclaw

In [8]:
from clawpack.geoclaw import dtopotools

In [9]:
fault = dtopotools.Fault()
fault.subfaults = []
df.loc[0,'lat']

for index, row in df.iterrows():
#     row = df.loc[0]
    usgs_subfault = dtopotools.SubFault()
    usgs_subfault.latitude = row['lat']
    usgs_subfault.longitude = row['lon']
    usgs_subfault.depth = row['depth']
    usgs_subfault.slip = row['slip']
    usgs_subfault.strike = row['strike']
    usgs_subfault.dip = row['dip']
    usgs_subfault.rake = row['rake']
    usgs_subfault.length = row['L']
    usgs_subfault.width = row['W']
    usgs_subfault.coordinate_specification = 'centroid'
    fault.subfaults.append(usgs_subfault)

In [10]:
fault.Mw()

8.2678497400625535

In [ ]:
x = np.linspace(139,146,360)
y = np.linspace(35,41,361)
# x = np.linspace(-120,-60,361)
# y = np.linspace(-60,0,361)

times = [1.]
fault.create_dtopography(x,y,times)

In [ ]:
dtopo_fname='usgs.tt3'
fault.dtopo.write(dtopo_fname, dtopo_type=3)

In [ ]:
import os
plt.figure(figsize=(12,7))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)
fault.plot_subfaults(axes=ax1,slip_color=True)
ax1.set_xlim(139,146)
ax1.set_ylim(35,41)
ax2.set_xlim(139,146)
ax2.set_ylim(35,41)
fault.dtopo.plot_dZ_colors(1.,axes=ax2, cmax_dZ=5.0, dZ_interval=10)
fname = os.path.splitext(os.path.split(dtopo_fname)[-1])[0] + '.png'
plt.savefig(fname)
print("Created ",fname)


In [ ]:
fault.dtopo.plot_dZ_colors(1.0, cmax_dZ=10.0, dZ_interval=100)
plt.gca().set_xlim(139,146)
plt.gca().set_ylim(35,41)

# Easywave

In [ ]:
df['reference']='C'
df['depth'] /=1000
df['L'] /=1000
df['W'] /=1000

In [ ]:
f = open('fault.flt','w')

for index, row in df.iterrows():
    rowString = '-location {lon} {lat} {depth} -refpos {reference} -strike {strike} -dip {dip} -rake {rake} -slip {slip} -size {L} {W}\n'
    rowString = rowString.format(**row)
    f.write(rowString)

f.close()